# Pre-Installation of saved data and Libraries Requires

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import time
import pickle
import numpy as np
import cv2
import pickle
from matplotlib import pyplot as plt
from keras import applications
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras import applications
from keras.models import Model
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [3]:
binary_model=pickle.load(open('/content/drive/MyDrive/Capstone_606/Features/Models/new_binary_class_vgg19model_svm.sav', 'rb'))
multi_model=pickle.load(open('/content/drive/MyDrive/Capstone_606/Features/Models/new_multi_class_vgg19model_vgg19_svm.sav', 'rb'))

# Implementation of models loaded to extract final results





In [4]:
def img_preProcess(img):
  pp1=image.img_to_array(img)
  pp1=np.expand_dims(pp1, axis=0)
  pp1=preprocess_input(pp1)
  return pp1

In [ ]:
# To chnage frame rate of a video 1 fps
# ffmpeg -i Umpire_Signal.mp4 -r 1 output2.mp4

In [5]:
import cv2
def video_stats(video):
  cam = cv2.VideoCapture(video)
  fps = cam.get(cv2.CAP_PROP_FPS)
  print("frame rate" ,int(fps),"FramesPerSecond")
  print("Duration of the video",cam.get(cv2.CAP_PROP_FRAME_COUNT)/fps)

In [6]:
video_stats('/content/drive/MyDrive/Capstone_606/Data/Out_six.mp4')

frame rate 25 FramesPerSecond
Duration of the video 31.24640625


In [7]:
import os
os.getcwd()

'/content'

In [8]:
os.mkdir('/content/data_frame_demo')

Breaking video into frames and save them in temporary directory

In [9]:
video_input='/content/drive/MyDrive/Capstone_606/Data/Out_six.mp4'

In [10]:
import cv2
import numpy as np
import os
frame_list=[]
c=0
vid_cap = cv2.VideoCapture(video_input)
while (vid_cap.isOpened()):
  ret, frame = vid_cap.read()
  if ret == False:
    break
  c=c+1
  cv2.imwrite('/content/data_frame_demo/'+"%d.jpg" % c, frame)
vid_cap.release()
cv2.destroyAllWindows()

In [11]:
print("Number of frames captured",len(os.listdir('/content/data_frame_demo')))

Number of frames captured 706


In [12]:

import cv2
import pandas as pd
time_dict={}
noball=[]
out=[]
six=[]
wide=[]
noaction=[]
cam = cv2.VideoCapture(video_input)
fps = cam.get(cv2.CAP_PROP_FPS)
base_model = applications.vgg19.VGG19(include_top=True, weights='imagenet', input_tensor=None,
                                      input_shape=None, pooling=None, classes=1000)
model_load_path='/content/drive/MyDrive/Capstone_606/Features'

model = Model(base_model.input, base_model.get_layer('fc1').output)
binary_model = pickle.load(open('/content/drive/MyDrive/Capstone_606/Features/Models/new_binary_class_vgg19model_svm.sav', 'rb'))
multi_model = pickle.load(open('/content/drive/MyDrive/Capstone_606/Features/Models/new_multi_class_vgg19model_vgg19_svm.sav', 'rb'))

frame_num=2*fps
frame_num=int(frame_num)
while(frame_num<len(os.listdir('/content/data_frame_demo'))):
  img = cv2.imread('/content/data_frame_demo/'+str(frame_num)+'.jpg')
  img2 = cv2.resize(img, (224, 224))
  frame_num=frame_num+int(fps/2)
  x=img_preProcess(img2)
  features = model(x) 
  predicted_values = binary_model.predict(features)
  if predicted_values==[2]:
    print("Umpire Found")
    predicted_values_2 = multi_model.predict(features)
    choices = {'1':'noball', '2':'out', '3':'six', '4':'wide', '5':'noaction'}
    result = choices.get(np.str(int(predicted_values_2)), 'default')
    if result == 'noball':
        noball.append(frame_num)
        #time_dict[count]='noball'
    elif result == 'out':
      out.append(frame_num)
        #time_dict[count]='out'
    elif result == 'six':
      six.append(frame_num)
        #time_dict[count]='six'
    elif result == 'wide':
      wide.append(frame_num)
        #time_dict[count]='wide'
    elif result == 'noaction':
      noaction.append(frame_num)
        #time_dict[count]='noaction'
time_dict["No_Ball"]=noball
time_dict["Out"]=out
time_dict["Six"]=six
time_dict["Wide"]=wide
time_dict["No_Action"]=noaction
summary_df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in time_dict.items() ]))
cv2.destroyAllWindows()

574717952/574710816 [==============================] - 9s 0us/step
Umpire Found
Umpire Found
Umpire Found
Umpire Found
Umpire Found
Umpire Found
Umpire Found
Umpire Found
Umpire Found


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [13]:
summary_df.head()

,No_Ball,Out,Six,Wide,No_Action
0,NaN,158,542.0,NaN,NaN
1,NaN,170,554.0,NaN,NaN
2,NaN,506,566.0,NaN,NaN
3,NaN,518,578.0,NaN,NaN
4,NaN,530,NaN,NaN,NaN


In [14]:
df_test10=summary_df

In [15]:
def to_seconds(df,fps):
  dict_sec={}
  time_frames_list=[]
  for i in df.columns:
    if df[i].isna().sum()==df.shape[0]:
      df=df.drop(columns=i)
    else:
      df[i]=df[i].dropna()
      df[i]=[y/fps for y in df[i]]
      df[i]=df[i].fillna(0)
      df[i]=[int(x) for x in df[i]]
      df[i]=[x-4 if x!=0 else 0 for x in df[i]] 
      df=df.drop_duplicates()
      test_list = list(set(list(df[i])))
      dict_sec[i]=test_list
      time_frames_list=time_frames_list+test_list
      time_frames_list = list(set(time_frames_list))
  for i in range(3,len(time_frames_list)):
    if time_frames_list[i-3]+3==time_frames_list[i]:
      del time_frames_list[i-2:i]
  return time_frames_list #dict_sec

In [16]:
cam = cv2.VideoCapture(video_input)
fps = cam.get(cv2.CAP_PROP_FPS)
cut_list=to_seconds(df_test10,fps)

In [17]:
del cut_list[0]
del cut_list[-1]

In [18]:
cut_list

[2, 16]

In [23]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
for j in range(0,len(cut_list)):
  ffmpeg_extract_subclip(video_input, cut_list[j]-1,cut_list[j] +5, targetname='/content/'+str(j)+'Highlight'+'.mp4')


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/drive/MyDrive/Capstone_606/Data/Out_six.mp4 -ss 1.00 -t 6.00 -vcodec copy -acodec copy /content/0Highlight.mp4
... command successful.

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/drive/MyDrive/Capstone_606/Data/Out_six.mp4 -ss 15.00 -t 6.00 -vcodec copy -acodec copy /content/1Highlight.mp4
... command successful.
